<a href="https://colab.research.google.com/github/PerfectionistAF/CVD_Risk/blob/main/StageOneTables_GP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# *Data Analysis: Tabular Data*

#*Pretraining: Tabular Data*